### packages

In [5]:
import pandas as pd

import numpy as np

import geopandas as gp

from shapely.geometry import Point, Polygon

import networkx as nx

import folium

import h3

import math

import random

import pulp

import copy

import os, sys

from sklearn.preprocessing import normalize

import warnings

warnings.filterwarnings('ignore')

from scipy.stats import poisson

import scipy.stats as stats

def Truncated_Gauss(mu,sigma,lower,upper):

    X = stats.truncnorm((lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma)
    
    return round(X.rvs(1)[0],2)


### object

In [6]:
class Order(object):
    
    def __init__(self,info):
        
        self.Pickup_Latitude=info.Pickup_Latitude
        
        self.Pickup_Longitude=info.Pickup_Longitude
        
        self.Dropoff_Latitude=info.Dropoff_Latitude
        
        self.Dropoff_Longitude=info.Dropoff_Longitude
        
        self.Pickup_Grid=info.Pickup_Grid
        
        self.Dropoff_Grid=info.Dropoff_Grid
        
        self.Fare=info['Fare($)']
        
        self.Matching_patience=info['Matching_patience(10s)']
        
        self.Travel_time=info['Travel_time(10s)']
        
class Driver(object):
    
    def __init__(self,info):
        
        self.Latitude=float(info.Point.replace('(','').replace(')','').split(',')[0])
        
        self.Longitude=float(info.Point.replace('(','').replace(')','').split(',')[1])
        
        self.beta_0=info.beta_0
        
        self.beta_1=info.beta_1
        
        self.beta_2=info.beta_2
        
        self.beta_3=info.beta_3
        
        self.u_c=info.u_c

### methods

In [7]:
class Global(object):
    
    def __init__(self,utility,decline,pickup_array):
        
        self.utility=utility
        
        self.decline=decline
        
        self.x_opt=self.inform()
    
    def inform(self):
        
        order_num=self.utility.shape[0]
        
        driver_num=self.utility.shape[1]
        
        x = np.ones([order_num,driver_num])
        
        return x
    
class Local(object):
    
    def __init__(self,utility,decline,pickup_array):
        
        self.utility=utility
        
        self.decline=decline
        
        self.pickup_array=pickup_array
        
        self.x_opt=self.inform()
    
    def inform(self):
        
        order_num=self.utility.shape[0]
        
        driver_num=self.utility.shape[1]
        
        x = np.zeros([order_num,driver_num])
        
        for o in range(order_num):
        
            for d in range(driver_num):
                
                if self.pickup_array[o][d]<5.92:
                    
                    x[o][d]=1
        
        return x
    
class Dispatch(object):
    
    def __init__(self,utility,decline,pickup_array):
        
        self.utility=utility
        
        self.decline=decline

        self.pickup_array=pickup_array
        
        self.x_opt=self.matching()
    
    def matching(self):
        
        order_num=self.utility.shape[0]
        
        driver_num=self.utility.shape[1]



        x = np.zeros([order_num,driver_num])
        
        # model

        model = pulp.LpProblem("Ride_Matching_Problems", pulp.LpMaximize)

        # variables

        X = pulp.LpVariable.dicts("X",((O,D) for O in range(order_num) for D in range(driver_num)),lowBound=0,upBound=1,cat='Integer')

        # objective

        # model += (pulp.lpSum([(self.utility[O][D]/(self.utility[O][D]+self.decline[D])) * X[(O, D)] for O in range(order_num) for D in range(driver_num)]))

        model += (pulp.lpSum([(1/self.pickup_array[O][D]) * X[(O, D)] for O in range(order_num) for D in range(driver_num)]))
    
        # constraints

        for O in range(order_num):

            model += pulp.lpSum([X[(O, D)] for D in range(driver_num)]) <= 1

        for D in range(driver_num):

            model += pulp.lpSum([X[(O, D)] for O in range(order_num)]) <= 1
        
        # solvable

        model.solve()
        
        for var in X:

            var_value = X[var].varValue
            
            if var_value !=0:
                
                x [var[0]][var[1]]=1

        
        return x

### parameters

In [8]:
# Path

Demand_path='./data/demand/'

Supply_path='./data/supply/'

Save_path='./data/result/'

# Time stamp 

Start_step=6120 

End_step=7560

# Preference parameters

beta_0,sigma_0 = 0.1,0.1

beta_1,sigma_1 = 1.0,0.1

beta_2,sigma_2 = 0.4,0.1

beta_3,sigma_3 = 1.0,0.1

u_c,sigma_c = 30.0,1

# Speed

speed=20000/60

# Spatial value

Spatial_value=dict(np.load(os.path.join(Demand_path,'Spatial_value.npy'),allow_pickle=True)[()])

Strategy='Dispatch'

### Specify the dates

dt='2020-12-01'

### simulation

In [299]:
# trip orders

trip_df=pd.read_csv(os.path.join(Demand_path,'Trip_Order_df_'+str(dt)+'.csv'))

trip_df=trip_df.drop(columns=['Unnamed: 0'])

# food orders

food_df=pd.read_csv(os.path.join(Demand_path,'Food_Order_df_'+str(dt)+'.csv'))

food_df=food_df.drop(columns=['Unnamed: 0'])

# merge orders

order_df=pd.concat([trip_df,food_df],ignore_index=True)

order_df=order_df.sort_values(by=['Arrive_step'])

order_df['Informs']=0

order_df['Responses']=0

order_df=order_df.reset_index(drop=True)

# avaiable drivers

driver_df=pd.read_csv(os.path.join(Supply_path,'Driver_df_'+str(dt)+'.csv'))

driver_df['beta_0']=driver_df.apply(lambda x:Truncated_Gauss(beta_0,sigma_0,beta_0-sigma_0,beta_0+sigma_0),axis=1)

driver_df['beta_1']=driver_df.apply(lambda x:Truncated_Gauss(beta_1,sigma_1, beta_1-sigma_1,beta_1+sigma_1),axis=1)

driver_df['beta_2']=driver_df.apply(lambda x:Truncated_Gauss(beta_2,sigma_2,beta_2-sigma_2,beta_2+sigma_2),axis=1)

driver_df['beta_3']=driver_df.apply(lambda x:Truncated_Gauss(beta_3,sigma_3,beta_3-sigma_3,beta_3+sigma_3),axis=1)

driver_df['u_c']=driver_df.apply(lambda x:Truncated_Gauss(u_c,sigma_c,u_c-sigma_c,u_c+sigma_c),axis=1)

driver_df['Menus']=0

driver_df['Selection']='decline'

driver_df=driver_df.drop(columns=['Unnamed: 0'])

driver_df

print('*'*100)

print('Demand size is ',order_df.shape[0])

print('Supply capacity is ',driver_df.shape[0])


### 02 Simulation: rolling time optimization

for step in range(Start_step,End_step+1,1):

    # 02-1 collect active orders

    order_batch=order_df[(order_df['Arrive_step']<=step)&(order_df['Driver_id']=='Waiting')]

    order_info={}

    for idx,row in order_batch.iterrows():

        order_info[row['Order_id']]=Order(row)

    orders=list(order_info.keys())

    # 02-2 collect avaiable drivers

    driver_batch=driver_df[(driver_df['Step']==step)&(driver_df['Order_id']=='Idle')]

    driver_info={}

    for idx,row in driver_batch.iterrows():

        driver_info[row['Driver_id']]=Driver(row)

    drivers=list(driver_info.keys())

    # 02-3 calculate the assignment utilities

    order_num=len(orders)

    driver_num=len(drivers)

    type_array=1*np.frompyfunc(lambda x: 'T' in x, 1, 1)(np.array(order_batch.Order_id))

    type_array=np.transpose([type_array] * driver_num)

    price_array=np.array(order_batch['Fare($)'])

    price_array=np.transpose([price_array] * driver_num)*5

    pickup_array=np.zeros([order_num,driver_num])

    spatial_array=np.zeros(order_num)

    for o in range(order_num):

        for d in range(driver_num):

            order_id,driver_id=orders[o],drivers[d]

            origin=Point(order_info[order_id].Pickup_Latitude,order_info[order_id].Pickup_Longitude)

            location=Point(driver_info[driver_id].Latitude,driver_info[driver_id].Longitude)

            pickup_array[o][d]=(origin.distance(location)*111000*1.2)/speed

    for o in range(order_num):

        order_id=orders[o]

        dest_grid=order_info[order_id].Dropoff_Grid

        spatial_array[o]=Spatial_value[dest_grid]

    spatial_array=np.transpose([spatial_array] * driver_num)

    utility=beta_0*type_array+beta_1*price_array-beta_2*pickup_array+beta_3*spatial_array

    # utility=price_array

    decline=np.ones(driver_num)*u_c

    # 02-4 Assignment

    assignment=eval(Strategy)(utility,decline,pickup_array).x_opt

    for o in range(order_num):

        order_id=orders[o]

        driver_informs=np.nonzero(assignment[o,:])[0]

        if len(driver_informs)!=0:

            order_df.loc[(order_df['Order_id']==order_id),'Informs']=len(driver_informs)


    # 02-5 Selection

    selection=np.zeros([order_num,driver_num])

    for d in range(driver_num):

        driver_id=drivers[d]

        order_menus=np.nonzero(assignment[:,d])[0]

        if len(order_menus)!=0:

            driver_df.loc[(driver_df['Step']==step)&(driver_df['Driver_id']==driver_id),'Menus']=len(order_menus)

            personal_utilities={}

            personal_utilities[-1]=driver_info[driver_id].u_c

            for o in order_menus:

                order_id=orders[o]

                types=type_array[o][d]

                fare=price_array[o][d]

                pickup=pickup_array[o][d]

                spatial=spatial_array[o][d]

                u=driver_info[driver_id].beta_0*types+driver_info[driver_id].beta_1*fare-driver_info[driver_id].beta_2*pickup+driver_info[driver_id].beta_3*spatial

                personal_utilities[o]=u

            # selection

            items=list(personal_utilities.keys())

            item_utilities=[u for u in list(personal_utilities.values())]

            exp_sum=sum([np.exp(u) for u in item_utilities])

            probabilities=[np.exp(u)/exp_sum for u in item_utilities]

            o=np.random.choice(items,p=probabilities)

            if o!=-1:

                selection[o][d]=1

                order_id,driver_id=orders[o],drivers[d]

                driver_df.loc[(driver_df['Step']==step)&(driver_df['Driver_id']==driver_id),'Selection']=order_id

    # 02-6 matching

    match={}

    for o in range(order_num):

        order_id=orders[o]

        response_drivers=np.nonzero(selection[o,:])[0]

        order_df.loc[(order_df['Order_id']==order_id),'Responses']=len(response_drivers)

        matrixs={}

        for d in response_drivers:

            matrixs[d]=pickup_array[o][d]

        if len(matrixs)!=0:

            d=min(matrixs, key=matrixs.get)

            match[o]=d

    # 02-7 update system

    for o,d in match.items():

        order_id,driver_id=orders[o],drivers[d]

        # orders

        order_df.loc[(order_df['Order_id']==order_id),'Driver_id']=driver_id

        order_df.loc[(order_df['Order_id']==order_id),'Response_step']=step

        order_df.loc[(order_df['Order_id']==order_id),'Pickup_step']=step+int(pickup_array[o][d]*6)

        driver_df.loc[(driver_df['Step']==step)&(driver_df['Driver_id']==driver_id),'Order_id']=order_id

        # drivers

        Added_item={}

        Added_item['Driver_id']=driver_id

        Added_item['Order_id']='Idle'

        Added_item['Step']=step+int(pickup_array[o][d]*6)+order_info[order_id].Travel_time

        Added_item['Point']='('+str(order_info[order_id].Dropoff_Latitude)+','+str(order_info[order_id].Dropoff_Longitude)+')'

        Added_item['Grid']=str(order_info[order_id].Dropoff_Grid)

        Added_item['Menus']=0

        Added_item['Selection']='decline'

        Added_item['beta_0']=driver_info[driver_id].beta_0

        Added_item['beta_1']=driver_info[driver_id].beta_1

        Added_item['beta_2']=driver_info[driver_id].beta_2

        Added_item['beta_3']=driver_info[driver_id].beta_3

        Added_item['u_c']=driver_info[driver_id].u_c

        driver_df=driver_df.append(Added_item, ignore_index=True)

    # 02-8 cancellation

    unmatched_orders=[o for o in list(range(order_num)) if o not in match.keys()]

    unmatched_orders=[orders[o] for o in unmatched_orders]

    if len(unmatched_orders)!=0:
        
        order_df.loc[((step-order_df['Arrive_step'])>order_df['Matching_patience(10s)'])&(order_df['Order_id'].isin(unmatched_orders)),'Driver_id']='Canceled'
        
    # 02-9 Quit

    unmatched_drivers=[d for d in list(range(driver_num)) if d not in match.values()]

    for d in unmatched_drivers:

        driver_id=drivers[d]

        df=copy.deepcopy(driver_df.loc[driver_df.Driver_id==driver_id])

        df=df.reset_index(drop=True)

        # if df.Selection.iloc[-60:].to_list().count('decline')==60:

        #     driver_df.loc[(driver_df['Step']==step+1)&(driver_df['Driver_id']==driver_id),'Order_id']='Force_Quit'

        if df.Order_id.iloc[-180:].to_list().count('Idle')==180:

            driver_df.loc[(driver_df['Step']==step)&(driver_df['Driver_id']==driver_id),'Order_id']='Quit'
        

        del df


    # 02-9 Parking

    repositioned_drivers=copy.deepcopy(driver_df.loc[(driver_df['Step']==step)&(driver_df['Order_id']=='Idle')])

    repositioned_drivers['Step']+=1

    driver_df=pd.concat([driver_df,repositioned_drivers],ignore_index=True)

    
    # 02-10 print

    print('\n'*2)

    print('*'*50)

    print('Current step: ',step)

    print('Current method: ',Strategy)

    print('Avaiable orders: ',len(orders))

    print('Avaiable drivers: ',len(drivers))
    
    print('Current selections: ',selection.sum())

    print('Current matched pairs: ',len(match))
    
    print('Current repositioning drivers: ',len(repositioned_drivers))

    print('Cumulative Match: ',order_df.loc[(order_df['Driver_id']!='Waiting')&(order_df['Driver_id']!='Canceled')].shape[0])

    print('Cumulative Cancellation: ',order_df.loc[(order_df['Driver_id']=='Canceled')].shape[0])

    print('Cumulative Response ratio: ',order_df.loc[(order_df['Driver_id']!='Waiting')&(order_df['Driver_id']!='Canceled')].shape[0]/order_df.shape[0])

    print('Utility average value: ',utility.mean())

    break
    

****************************************************************************************************
Demand size is  6085
Supply capacity is  712



**************************************************
Current step:  6120
Current method:  Dispatch
Avaiable orders:  3
Avaiable drivers:  505
Current selections:  3.0
Current matched pairs:  3
Current repositioning drivers:  502
Cumulative Match:  3
Cumulative Cancellation:  0
Cumulative Response ratio:  0.0004930156121610517
Utility average value:  36.674368548538915


In [266]:
order_df.to_csv(os.path.join(Save_path,'order_df_'+Strategy+'_'+str(dt)+'.csv'))

driver_df.to_csv(os.path.join(Save_path,'driver_df_'+Strategy+'_'+str(dt)+'.csv'))
